In [ ]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

In [ ]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("numpy")

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RecommenderSystem").getOrCreate()

In [ ]:
import numpy as np
from pyspark.sql.window import Window
from pyspark.sql.functions import expr, rank, col, udf
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, FloatType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.mllib.evaluation import RankingMetrics
from pyspark.sql import functions as F

In [5]:
df = spark.read.csv("sample-user-filtered-2023.csv", header=True, inferSchema=True)

In [ ]:
df = df.filter(df.rating != 0)

In [6]:
df = df.orderBy("user_id", "anime_id")

In [7]:
df.show()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      1|      19|     9|
|      1|      20|    10|
|      1|      21|     9|
|      1|     154|     8|
|      1|     199|    10|
|      1|     269|     7|
|      1|     442|     7|
|      1|     481|     8|
|      1|    1535|     9|
|      1|    1575|    10|
|      1|    1735|    10|
|      1|    2472|     7|
|      1|    4224|     9|
|      1|    5081|     8|
|      1|    5114|    10|
|      1|    6547|     6|
|      1|    7674|     7|
|      1|    9253|    10|
|      1|    9919|     8|
|      1|   10087|     7|
+-------+--------+------+
only showing top 20 rows



In [8]:
df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [9]:
user_window = Window.partitionBy("user_id").orderBy(col("anime_id").desc())
df = df.withColumn("num_animes", expr("count(*) over (partition by user_id)"))

In [10]:
percent_animes_to_mask = 0.3

df_final = df.withColumn("num_animes_to_mask", (col("num_animes") * percent_animes_to_mask).cast("int"))
df_final = df_final.withColumn("anime_rank", rank().over(user_window))

indexer_user = StringIndexer(inputCol='user_id', outputCol='user_index').setHandleInvalid("keep")
indexer_anime = StringIndexer(inputCol='anime_id', outputCol='anime_index').setHandleInvalid("keep")

user_indexer_model = indexer_user.fit(df_final)
anime_indexer_model = indexer_anime.fit(df_final)

df_final = user_indexer_model.transform(df_final)
df_final = anime_indexer_model.transform(df_final)

df_final = df_final.withColumn('user_index', df_final['user_index'].cast('integer'))
df_final = df_final.withColumn('anime_index', df_final['anime_index'].cast('integer'))

user_converter = IndexToString(inputCol='user_index', outputCol='user_id')
anime_converter = IndexToString(inputCol='anime_index', outputCol='anime_id')

user_converter.setLabels(user_indexer_model.labels)
anime_converter.setLabels(anime_indexer_model.labels)

train_data = df_final.filter(col("anime_rank") > col("num_animes_to_mask"))
test_data = df_final.filter(col("anime_rank") <= col("num_animes_to_mask"))

In [12]:
final_als = ALS(
    userCol='user_index',
    itemCol='anime_index',
    ratingCol='rating',
    coldStartStrategy='drop',
    nonnegative=True,
    rank=10,
    maxIter=20,
    regParam=0.15
)

best_model = als.fit(train_data)

In [13]:
predictions = best_model.transform(test_data)
predictions = predictions.withColumn("prediction", expr("CASE WHEN prediction < 1 THEN 1 WHEN prediction > 10 THEN 10 ELSE prediction END"))
evaluator = RegressionEvaluator(labelCol='rating', predictionCol='prediction')

metrics = {}

metric_names = ['rmse', 'mae', 'mse', 'r2']

for metric in metric_names:
    evaluator.setMetricName(metric)
    metrics[metric] = evaluator.evaluate(predictions)

print(f"Root Mean Squared Error (RMSE): {metrics['rmse']}")
print(f"Mean Absolute Error (MAE): {metrics['mae']}")
print(f"Mean Squared Error (MSE): {metrics['mse']}")
print(f"R² (coefficient of determination): {metrics['r2']}")

Root Mean Squared Error (RMSE): 1.3724798015808322
Mean Absolute Error (MAE): 1.057306571758675
Mean Squared Error (MSE): 1.8837008057473603
R² (coefficient of determination): 0.2511428250451453


In [ ]:
def evaluate_als_recommendations(model, test_data, k=10):
    user_subset = test_data.select('user_index').distinct()
    recommendations = model.recommendForUserSubset(user_subset, k)

    predicted_rankings = recommendations.rdd.map(
        lambda row: (row.user_index, [int(x.anime_index) for x in row.recommendations])
    )

    actual_rankings = test_data.groupBy('user_index').agg(
        F.collect_list('anime_index').alias('actual_items')
    ).rdd.map(
        lambda row: (row.user_index, [int(x) for x in row.actual_items])
    )

    prediction_truth = predicted_rankings.join(actual_rankings)

    metrics = RankingMetrics(prediction_truth.map(lambda x: (x[1][0], x[1][1])))

    # Calculate various metrics
    results = {
        'MAP': metrics.meanAveragePrecision,
        'NDCG at K': metrics.ndcgAt(k),
        'Precision at K': metrics.precisionAt(k),
        'Recall at K': metrics.recallAt(k)
    }

    return results

# Use the evaluation function
metrics_results = evaluate_als_recommendations(best_model, test_data, k=10)

# Print results
for metric_name, value in metrics_results.items():
    print(f"{metric_name}: {value:.4f}")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


MAP: 0.0219
NDCG at K: 0.1322
Precision at K: 0.1228
Recall at K: 0.0421


In [26]:
userRecs = best_model.recommendForAllUsers(10)
userRecs = user_converter.transform(userRecs)

def create_reverse_recommendations():
    anime_labels = anime_indexer_model.labels

    @udf(ArrayType(StructType([
        StructField("anime_id", StringType(), True),
        StructField("rating", FloatType(), True)
    ])))
    def convert_recommendations(recommendations):
        return [{"anime_id": str(anime_labels[int(rec.anime_index)]),
                "rating": float(rec.rating)}
                for rec in recommendations]

    return convert_recommendations

convert_recs = create_reverse_recommendations()
userRecs = userRecs.withColumn("recommendations", convert_recs(col("recommendations")))

In [ ]:
def format_recommendations_for_export(userRecs):
    flattened = userRecs.select(
        'user_id',
        F.posexplode('recommendations').alias('position', 'rec')
    ).select(
        'user_id',
        'position',
        'rec.anime_id',
        'rec.rating'
    )

    pivoted = flattened.groupBy('user_id').pivot('position').agg(
        F.first('anime_id').alias('anime_id'),
        F.first('rating').alias('predicted_rating')
    )

    for i in range(10):
        pivoted = (pivoted
            .withColumnRenamed(f'{i}anime_id', f'recommendation_{i+1}_anime_id')
            .withColumnRenamed(f'{i}_predicted_rating', f'recommendation_{i+1}_score'))

    final_df = pivoted.orderBy('user_id')

    return final_df

formatted_recs = format_recommendations_for_export(userRecs)

formatted_recs.toPandas().to_csv('anime_recommendations.csv', index=False)

formatted_recs.coalesce(1).write.mode('overwrite').option('header', 'true').csv('anime_recommendations_spark')

formatted_recs.show(5)

+-------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+-----------------------+
|user_id|0_anime_id|recommendation_1_score|1_anime_id|recommendation_2_score|2_anime_id|recommendation_3_score|3_anime_id|recommendation_4_score|4_anime_id|recommendation_5_score|5_anime_id|recommendation_6_score|6_anime_id|recommendation_7_score|7_anime_id|recommendation_8_score|8_anime_id|recommendation_9_score|9_anime_id|recommendation_10_score|
+-------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+------